In [1]:
import numpy as np
import cv2
import sys
np.set_printoptions(threshold=sys.maxsize)

In [2]:
cap = cv2.VideoCapture('./train/trainGray.mp4')
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

bufTemp = np.empty((frameCount, frameHeight * frameWidth), np.dtype('uint8'))

fc = 0
ret = True

while (fc < frameCount  and ret):
    ret, x = np.array(cap.read())
    x = x[:,:,0]
    x.shape = frameHeight*frameWidth
    bufTemp[fc] = x
    fc += 1

cap.release()

x = bufTemp[0, 0:frameHeight*frameWidth]
x.shape = (frameHeight, frameWidth)

In [16]:
y = np.loadtxt('./train/train.txt', np.dtype('float'))
theta = np.zeros(((frameHeight*frameWidth)+1), np.dtype('int16'))
m = frameCount-1
# alpha = [0,0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56,5.12,10.24]
# lmbda = [0,0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56,5.12,10.24]
alpha = [0.01,0.16,0.64,1.28]
lmbda = [0,0.01,0.16,0.64,5.12]
alpLmbTheta = np.zeros((len(alpha), len(lmbda), (frameHeight*frameWidth)+1), np.dtype('int16'))
alpLmbCost = np.zeros((len(alpha), len(lmbda)), np.dtype('float'))
loopCount = 2 # Was 5
costArr = np.zeros((loopCount), np.dtype('float'))

In [22]:
print(theta.shape)
print(alpLmbTheta.shape)
print(alpLmbTheta[0,0].shape)
print(alpLmbCost.shape)
# print(alpLmbTheta[0,0])

# for idx, alp in enumerate(alpha):
#     for idx2, lmbd in enumerate(lmbda):
#         print('%d:' % (idx), alp, '  %d:' % (idx2), lmbd)

(307201,)
(4, 5, 307201)
(307201,)
(4, 5)


In [29]:
for idx, alp in enumerate(alpha):
    for idx2, lmbd in enumerate(lmbda):
        for i in range(loopCount):
            cost = 0
            for j in range(1,2):
                firstFrame = np.array(bufTemp[j-1], dtype='int16')
                secondFrame = np.array(bufTemp[j], dtype='int16')
                x = np.empty(((frameHeight*frameWidth)+1), np.dtype('int16'))
                x[0] = 1
                x[1:(frameHeight*frameWidth)+1] = secondFrame - firstFrame
                x = np.absolute(x)
#                 print(x[0:100])
                mean = np.mean(x)
                x = x - mean
                x = x/np.std(x)
#                 print(x.shape)
#                 print(x[0:100])
                grad = ((np.dot(x, alpLmbTheta[idx, idx2]) - y[j])*x)/m
#                 print('1:',grad.shape)
#                 print(grad[0:100])
                grad[1:len(grad)] = grad[1:len(grad)] + ((lmbd/m)*alpLmbTheta[idx,idx2,1:len(alpLmbTheta[idx,idx2])])
#                 print('2:',grad.shape)
                cost = cost + ((np.dot(x, alpLmbTheta[idx, idx2]) - y[j])**2)/2
                alpLmbTheta[idx,idx2] = alpLmbTheta[idx,idx2] - (alp*grad)
            
            costArr[i] = cost/(2*m)
#         print(costArr)

[0.00035502 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502
 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502
 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502
 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502
 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502
 0.00035502 0.00035502 0.00035502 0.00021754 0.00021754 0.00021754
 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754
 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754
 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754
 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754
 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754 0.00035502
 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502
 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502
 0.00035502 0.00035502 0.00035502 0.00021754 0.00021754 0.00021754
 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754 0.0002

[0.00035502 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502
 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502
 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502
 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502
 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502
 0.00035502 0.00035502 0.00035502 0.00021754 0.00021754 0.00021754
 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754
 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754
 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754
 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754
 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754 0.00035502
 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502
 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502 0.00035502
 0.00035502 0.00035502 0.00035502 0.00021754 0.00021754 0.00021754
 0.00021754 0.00021754 0.00021754 0.00021754 0.00021754 0.0002

In [20]:
print(alpLmbTheta[0,2,:])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
for idx, alp in enumerate(alpha):
    for idx2, lmbd in enumerate(lmbda):
        for i in range(loopCount):
            cost = 0
            for j in range(1,frameCount):
                firstFrame = np.array(bufTemp[j-1], dtype='int16')
                secondFrame = np.array(bufTemp[j], dtype='int16')
                x = np.empty(((frameHeight*frameWidth)+1), np.dtype('int16'))
                x[0] = 1
                x[1:(frameHeight*frameWidth)+1] = secondFrame - firstFrame
                x = np.absolute(x)
                mean = np.mean(x)
                x = x - mean
                x = x/np.std(x)
                grad = ((np.dot(x, alpLmbTheta[idx, idx2]) - y[j])*x)/m
                grad[1:len(grad)] = grad[1:len(grad)] + ((lmbd/m)*alpLmbTheta[idx,idx2,1:len(alpLmbTheta[idx,idx2])])
                cost = cost + ((np.dot(x, alpLmbTheta[idx, idx2]) - y[j])**2)/2
                alpLmbTheta[idx,idx2] = alpLmbTheta[idx,idx2] - (alp*grad)
            
            costArr[i] = cost/(2*m)
        print(costArr)


In [ ]:
result = np.empty((len(alpha), len(lmbda), frameCount), np.dtype('float'))
                  
for idx in range(len(alpha)):
    for idx2 in range(len(lmbda)):
        for j in range(1,frameCount):
            firstFrame = np.array(bufTemp[j-1], dtype='int16')
            secondFrame = np.array(bufTemp[j], dtype='int16')
            x = np.empty(((frameHeight*frameWidth)+1), np.dtype('int16'))
            x[0] = 1
            x[1:(frameHeight*frameWidth)+1] = secondFrame - firstFrame
            x = np.absolute(x)
            mean = np.mean(x)
            x = x - mean
            x = x/np.std(x)
#             result[idx][idx2][j-1] = np.dot(x,theta)
            result[idx][idx2][j-1] = np.dot(x,alpLmbTheta[idx,idx2])
    
# result.tofile('./train/train2.txt',sep="\n",format="%s")

In [ ]:
# cost2 = 0

for idx in range(len(alpha)):
    for idx2 in range(len(lmbda)):
        for j in range(1,frameCount):
#             cost2 = cost2 + (((result[j-1] - y[j])**2)/2)
            alpLmbCost[idx,idx2] = alpLmbCost[idx,idx2] + (((result[idx][idx2][j-1] - y[j])**2)/2)

        # cost2 = cost2/(2*m)
        # print(cost2)
        alpLmbCost[idx,idx2] = alpLmbCost[idx,idx2]/(2*m)
        print(alpLmbCost[idx,idx2])

In [ ]:
print(alpLmbTheta.shape)
print(alpLmbCost.shape)

In [ ]:
alpLmbTheta[4][4]